In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd


In [ ]:
filename = 'fock_prep'
df = pd.read_csv(filename)
df

In [ ]:
def convert_complex_string_to_float(stringy):
    '''
    Convert '(a+0j)' to a where a,b are floats
    '''
    a = stringy.split('+')
    b = a[0].split('(')
    return float(b[1])
    
    

In [ ]:
import matplotlib
font = {'family' : 'normal',
        'weight' : 'light',
        'size'   : 13}

matplotlib.rc('font', **font)

In [ ]:
total_tasks = 5

fig = plt.figure(figsize = (6,5))

for i in range(1, total_tasks+1): 
    #each task
    
    #qutip fids
    #qutip_fids = list(df.loc[df['task'] ==i]['qutip_fid'])
    #qutip_fids = [1- convert_complex_string_to_float(i) for i in qutip_fids]
    bo_fids = list(df.loc[df['task'] ==i]['BO_fid'])
    bo_fids = [1-i for i in bo_fids]
    
    if len(bo_fids)==0:
        continue
    
    plt.plot(
        list(df.loc[df['task'] ==i]['layer']),
        bo_fids,
        marker = 'o', linestyle = '--' , label = str(i)      
        )
    #plt.plot(times, mean_buckets)

    
    #plt.errorbar(x, y, yerr = err, ecolor='black', capsize=10)#, label ='Line4')
#plt.text(8, 13, r'$|g>|n0> \rightarrow |g>|0n>$', fontsize = 15)
plt.grid(alpha=0.5, linestyle=':')

plt.xlabel('Depth', fontsize = 15)
#plt.hline(ecd_only)
plt.ylabel(r'$1 - \mathcal{F}$', fontsize = 20)
#plt.title('Beam Splitters', fontsize = 15)
plt.yscale('log')
#fig.savefig('Ancilla Relaxation')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0., fontsize = 13)
plt.text(4,4, ' $|gn0> \longrightarrow |g0n> $', fontsize = 15)
plt.tight_layout()

In [ ]:
fig.savefig('state_transfer_depth.png', dpi = 1000)

# DUMP

In [ ]:
conditions = ['bo_fidelity', 'only ecd', 'bare qubit mode coupling','stark shift', 'mode coupling', 'transmon relax','transmon dephasing', 'cavities relax', 'cavities dephasing']
x_pos = np.arange(len(conditions))
bars = []
bars_err = []
for i in range(5):
    listy = list(df[conditions[i]]) 
    
    #if complex strings
    if type(listy[0]) is not float:
        listy = [convert_complex_string_to_float(i) for i in listy]
    bars.append(1 - np.mean(listy))
    bars_err.append(np.std(listy))
    
#master equation solver fidelity err -- returns density matrix instead of state
for i in range(5, len(conditions)):
    listy = list(df[conditions[i]]) 
    listy = [np.sqrt(i) for i in listy]
    bars.append(1 - np.mean(listy))
    bars_err.append(np.std(listy))

fig, ax = plt.subplots(figsize = (8,6))
ax.bar(conditions, bars, yerr=bars_err, align='center', alpha=0.5, ecolor='black', capsize=10)
ax.set_ylabel('$1-\mathcal{F}$')
#ax.set_xticks(x_pos)
ax.set_xticklabels(conditions, fontsize = 15)
ax.set_title('Qutip Noise Simulation of g01 -> g10')
ax.yaxis.grid(True)
plt.xticks(range(len(conditions)), conditions, rotation = 90)

In [ ]:
#cond_labels =  ['Batch Optimizer', 'ECD Unitary Only', 'Dispersive ($\chi a^\dagger a \sigma_z $)',r'Stark Shift ($\chi |\alpha|^2  \sigma_z$)', 'Cross Kerr ($\chi_{ab} a^\dagger a b^\dagger b +...$)']

In [ ]:
#conditions = ['bo_fidelity', 'only ecd', 'bare_qubit_mode_coupling','Stark Shift', 'Mode Coupling']#, 'transmon_relax','transmon_dephasing']
#x_pos = np.arange(len(conditions))
# bars = []
# bars_err = []
# for i in range(len(conditions)):
#     listy = list(df[conditions[i]]) 
#     if i is not 0:     # for all master eq sim, fidelity is squared bc of overlap issue
#         listy = [np.sqrt(i) for i in listy]
#     bars.append(1 - np.mean(listy))
#     bars_err.append(np.std(listy))
# conditions[-1] = "Mode-Mode Coupling"
fig, ax = plt.subplots(figsize = (10,6))
ax.barh(conditions, bars, xerr=bars_err, align='center', alpha=0.5, ecolor='black', capsize=10)
ax.set_xlabel('$1-\mathcal{F}$', fontsize = 20)
#a#x.set_xticks(x_pos)
ax.set_yticklabels(conditions, fontsize = 15)
ax.set_title('Decoherence free error budget of g01 -> g10', fontsize = 20)
ax.xaxis.grid(True)
plt.yticks(range(len(conditions)), conditions)#, rotation = 60)
plt.tight_layout()
fig.savefig('Decoherence free error budget')
# colors = {'fruit':'red', 'veggie':'green'}         
# labels = list(colors.keys())

# handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
# plt.legend(handles, labels)

In [ ]:
len(df)

In [ ]:
conditions = ['only ecd', 'bare qubit mode coupling','stark shift', 'mode coupling', 'transmon relax','transmon dephasing', 'cavities relax', 'cavities dephasing']
x_pos = np.arange(len(conditions))
bars = []
bars_err = []
for i in range(5):
    listy = list(df[conditions[i]]) 
    
    #if complex strings
    if type(listy[0]) is not float:
        listy = [convert_complex_string_to_float(i) for i in listy]
    listy = [df['bo_fidelity'][i] - listy[i] for i in range(1, len(df))]
    bars.append(np.mean(listy))
    bars_err.append(np.std(listy))
    
#master equation solver fidelity err -- returns density matrix instead of state
for i in range(5, len(conditions)):
    listy = list(df[conditions[i]]) 
    listy = [np.sqrt(i) for i in listy]
    listy = [df['bo_fidelity'][i] - listy[i] for i in range(1, len(df))]
    
    bars.append( np.mean(listy))
    bars_err.append(np.std(listy))

fig, ax = plt.subplots(figsize = (8,6))
ax.bar(conditions, bars, yerr=bars_err, align='center', alpha=0.5, ecolor='black', capsize=10)
ax.set_ylabel('$1-\mathcal{F}_{opt}$')
#ax.set_xticks(x_pos)
ax.set_xticklabels(conditions, fontsize = 15)
ax.set_title('Qutip Noise Simulation of g01 -> g10')
ax.yaxis.grid(True)
plt.xticks(range(len(conditions)), conditions, rotation = 90)

In [ ]:
#conditions = ['bo_fidelity', 'only ecd', 'bare_qubit_mode_coupling','Stark Shift', 'Mode Coupling']#, 'transmon_relax','transmon_dephasing']
#x_pos = np.arange(len(conditions))
# bars = []
# bars_err = []
# for i in range(len(conditions)):
#     listy = list(df[conditions[i]]) 
#     if i is not 0:     # for all master eq sim, fidelity is squared bc of overlap issue
#         listy = [np.sqrt(i) for i in listy]
#     bars.append(1 - np.mean(listy))
#     bars_err.append(np.std(listy))
# conditions[-1] = "Mode-Mode Coupling"
fig, ax = plt.subplots(figsize = (10,6))
ax.barh(conditions, bars, xerr=bars_err, align='center', alpha=0.5, ecolor='black', capsize=10)
ax.set_xlabel('$1-\mathcal{F}_{opt}$', fontsize = 20)
#a#x.set_xticks(x_pos)
ax.set_yticklabels(conditions, fontsize = 15)
ax.set_title(' g01 -> g10', fontsize = 20)
ax.xaxis.grid(True)
plt.yticks(range(len(conditions)), conditions)#, rotation = 60)
plt.tight_layout()
#fig.savefig('Decoherence free error budget')
# colors = {'fruit':'red', 'veggie':'green'}         
# labels = list(colors.keys())

# handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
# plt.legend(handles, labels)

In [ ]:
bars

# Transmon Relaxation

In [ ]:
f2_name = 'Noise_sim_one_by_one__15_level_aug_11_transmon_relaxation'
df2 = pd.read_csv(f2_name)

In [ ]:
df2 = df2.drop(0)

In [ ]:
df2

In [ ]:
c = 0
for i in range(12):
    inconsis =np.abs(ast.literal_eval(df2['tramsmon_relaxation [50 us, 100 us, ...]'][i+1])[-5] - df['only ecd'][i+1])
    print(inconsis)
    c+=float(inconsis)
print('err on average is ' + str(c/12))

In [ ]:
i = 0
df2['tramsmon_relaxation [50 us, 100 us, ...]'][i+1][-1]

In [ ]:
len(df2)

In [ ]:
ecd_only = np.mean(df['only ecd'])
ecd_err = np.std(df['only ecd'])
ecd_err

In [ ]:
k = 1
j = 1
df2['tramsmon_relaxation [50 us, 100 us, ...]'][k+1]

In [ ]:
import ast
buckets = [[] for _ in range(len(ast.literal_eval(df2['tramsmon_relaxation [50 us, 100 us, ...]'][1])))]
for j in range(len(buckets)):# for each time
    for k in range(len(df)):
        #print(k)
        listy = ast.literal_eval(df2['tramsmon_relaxation [50 us, 100 us, ...]'][k+1])
        listy = [np.sqrt(i) for i in listy]
        buckets[j].append(-1*(listy[j] - np.sqrt(df['only ecd'][k+1])))

        

In [ ]:
mean_buckets = [np.mean(i) for i in buckets]
err_buckets = [np.std(i) for i in buckets]

In [ ]:
times = [50 + 50*i for i in range(10)]

In [ ]:
x = np.array(times)
y = np.array(mean_buckets)
err = np.array(err_buckets)

In [ ]:
import matplotlib
font = {'family' : 'normal',
        'weight' : 'light',
        'size'   : 10}

matplotlib.rc('font', **font)

In [ ]:
#plt.plot(times, mean_buckets)
fig = plt.figure(figsize = (10, 6))
#plt.errorbar(x, y, yerr = err, ecolor='black', capsize=10)#, label ='Line4')
plt.plot(x, y, 'k', marker = 'o', color='#CC4F1B')
plt.fill_between(x, y-err, y+err, alpha = 0.2)
plt.grid(alpha=0.5, linestyle=':')
plt.axhline(y=0, color='black', linestyle='--', label = 'No Noise')
plt.xlabel('Ancilla $T_1$ ($\mu s$)', fontsize = 15)
#plt.hline(ecd_only)
plt.ylabel(r'$\mathcal{F}_{true} - \mathcal{F}_{noisy}$', fontsize = 20)
plt.title(r'Ancilla Relxation for $|g01> \rightarrow |g10>$', fontsize = 15)
fig.savefig('Ancilla Relaxation')
#plt.legend()

In [ ]:
y.shape()

In [ ]:
#error = np.random.normal(0.1, 0.02, size=y.shape) +.1
y += np.random.normal(0, 0.1, size=y.shape)

pl.plot(x, y, 'k', color='#CC4F1B')
pl.fill_between(x, y-err, y+err,
    alpha=0.5, edgecolor='#CC4F1B', facecolor='#FF9848')